# Introduction

In lab1 you learned how the robot is controlled with ROS2 and how to run and tune a basic PID controller. Now that you are familar with this ROS based interface, we wrap everything into two easy-to-use `RobotSim` and `RobotReal` classes so you can focuse on how to make the robot do interesting things while these two classes take care of the ROS2 in the background. 

The goal of this notebook is to get you familiar with this new interface and and guid you along the installation process on your laptop so you can simulate the robot on your own when you're not in the lab. By the end of this notebook, you should be able to:

- Start the robot simulator on your laptop and read its states and send joint commands
- Launch the robot bridge on the hardware and communicate with it with your laptop! 
- Visualize the real robot's motion in the simulator app. 
- Implement basic control loops and record datasets of robot's state. 

Let's start!

# Installation Guide

First, you need to have Anaconda (or miniconda) installed on your laptop. After installing it, open the conda terminal and run the following command to create a virtual environment and install the lab's repository:

```bash 
cd path/to/NYU_ROB_2004
git pull # Make sure you synchrionize your fork with the master branch before pulling to get the latest updates
conda create --name rob2004 python=3.10 # When prompted, accept the terms 
conda activate rob2004
pip install -e . # Make sure you are in the repositorie's root directory before running this
```
If everything is done correctly, you should now have the lab's software stack installed!



# Launching the Simulator App

Now that you have the rob2004 installed on your laptop, we should be able to simulate the popper in MuJoCo. First, check for the simulator app installation by running the following in your terminal:

```bash
conda activate rob2004
rob2004-simulator --help
```

If the application is installed correctly, you'll get an output like the following:

```bash
usage: rob2004-simulator [-h] [--visualizer] [--fixed_body] [--port PORT] [--ip IP]

ZeroRPC Popper Mujoco simulator and visualizer.

options:
  -h, --help    show this help message and exit
  --visualizer  Visualizer mode (Physics disabled)
  --fixed_body  Fix the robot's body in the world.
  --port PORT   The network port on which the simulator server listens
  --ip IP       The IP on which the simulator server listens
```

**Note**: If you use a Mac computer, the command above does not work. In Mac you need to run the simulator using `mjpython` instead of the normal python interpreter. Do this by navigating to the repository directory and then runnign the following:

```bash
cd path/to/NYU_ROB_2004
conda activate rob2004
mjpython scripts/simulator_app.py --help
```

If everything is correct, you'll see the same help information as shown above. As printed in the help message, the simulator app has two important flags. 

- `--visualizer`: In this mode, the physics simulation is disabled and we use the app as a visualizer (for example mirror the real robot in the virtual world)
- `--fixed_body`: If you set this flag, the robot's body will be fixed in the world as if you have it sitting on the stand. In future labs you'll diactivate this flag to let the robot walk around in simulation!

In this notebook we will use the app as a visulizer of the current state of our real robot so launch the app as follows:

```bash
rob2004-simulator --fixed_body --visualizer # or mjpython scripts/simulator_app.py --fixed_body --visualizer 
```
If everthing is done right, you'll see a window like this:
<p align="center">
  <img src="assets/mujoco_sim.png" alt="image" width="50%" height="auto"/>
</p>

Also note that in the terminal, you'll see a message indicating that the simulator is running and waiting for you to connect to it:

```bash
Simulator is running on port 4242. Use RobotSim(port=4242) to communicate with the robot!
```

We use this information to connect to the simulator in our python scripts or Jupyter notebooks. First mport the `RobotSim`:

In [1]:
from rob2004.interface import RobotSim
robot_sim = RobotSim(port=4242)

Congrats! now you should be able to read the robot's states:

In [2]:
state = robot_sim.getJointState()
print(state)

{'q': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'dq': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'stamp': array(1.77051415e+09)}


You could also add visualization markers inside the scene for debugging purposes (you'll use this in the future):

In [ ]:
robot_sim.addMarker([0,0,0.3])

Finally, you can visualize the robot for a different set of joint states:

In [3]:
import numpy as np
q = np.zeros(12)
q[2] = 0.5
robot_sim.show(q)

**Your Task**: Use the visualizer above and identify the joints each of the 12 axis of the q vector control. Use this knowledge to complete the dictionary in the next cell (you'll need this later). Also report your finding in your final report. 

In [5]:
# Your Code:
joint_index_to_joint_name = {} # something like 0:front_right_1
joint_name_to_joint_index = {v:k for k,v in joint_index_to_joint_name.items()}


# Real Robot Bridge

## Starting the Bridge 
Now that you have the simulator, let's see how we can control the real robot! If you remember, in the previous lab you had to launch several ros nodes. The procedure here is very similar. First SSH into the robot:

```bash
ssh pi@robot_ip # Get the IP address of the robot the same way you did in Lab1
```

When logged in, navigate to the rob2004 repository on the robot:

```bash 
cd ~/rob2004
ros2 launch scripts/robot.launch.py
```
with running above, the robot's joints should initialize like what you saw in the previous lab. Now open anohter terminal and SSH into the robot again:
```bash
ssh pi@robot_ip 
rob2004-robot-bridge --port 4243
```
If successful, the robot is now ready and waiting for you to connect to it with the `RobotReal` class!

## Connecting to the Robot
We interact with the robot through a Python class with the exact same interface as the simulated robot. This way you'll be able to run your algorithms in simulation and easity transfer to the real robot when you come to the lab! Import the interface class and instantiate it. You'll need the IP address of your robot (the same address you used to SSH into it) for this:

In [ ]:
from rob2004.interface import RobotReal

robot_ip="" #Enter your robot's IP here
robot_real = RobotReal(ip=robot_ip, port=4243)

If everything is done right, you should be able to read the robot's joint states exactly like before:

In [ ]:
state = robot_sim.getJointState()
print(state)

**Your Task**: Run the commands above repeatedly in a loop and see the joint values (`q`) changing as you move the robot's joints.

In [2]:
import time  # Important: use time.sleep(0.1) to wait for 100 ms in between each update
# ToDo: Write a loop to print the joint positions q for 100 times

# Mirroring the Real Robot Movement In Simulation

Now that you have both the simulated and real robots running and can communicate with them (through `robot_sim` and `robot_real`), we can do a fun experiment. You can now write a similar loop to repeatedly show the latest status of the real robot in simulation:

In [ ]:
# ToDo: Complete this loop
import time

start_time = time.time()
while time.time()-start_time < 60:
    q = None # Replace with the joint position from the real robot
    # send the q above to the simulator
    time.sleep(0.1)

**Your Task**: Make a video of your success with this step and include it in your report.